В данном ноутбуке можно загрузить сохраненную модель, проверить качество на валидационном датасете и получить файл с предсказаниями для тестового датасета `submission.csv`

# RoBERTa-large

Для решения задачи я использовала предобученную модель **RoBERTa** из библиотеки **Hugging Face**.

Подробнее о модели можно прочитать по ссылкам:


*   [RoBERTa: A Robustly Optimized BERT Pretraining Approach](https://arxiv.org/abs/1907.11692)

*   [RoBERTa-large](https://huggingface.co/roberta-large)



## Dependencies

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Установим библиотеки от Hugging Face

In [2]:
!pip install git+https://github.com/huggingface/transformers
!pip install -U nlp

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-x6fjoo1v
  Running command git clone -q https://github.com/huggingface/transformers /tmp/pip-req-build-x6fjoo1v
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 1.1MB 3.4MB/s 
     |████████████████████████████████| 890kB 8.5MB/s 
     |████████████████████████████████| 3.0MB 8.9MB/s 
  Created wheel for transformers: filename=transformers-3.3.1-cp36-none-any.whl size=1061349 sha256=0614d9d433c938ee33933991d585764c32fe511ebd8168691040b8b0f4fc3cf7
  Stored in directory: /tmp/pip-ephem-wheel-cache-mq54zigh/wheels/70/d3/52/b3fa4f8b8ef04167ac62e5bb2accb62ae764db2a378247490e
Successfully built transformers
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=1cad5efd20261ff98755e44fb4795a8cbb316cdbdc2d21c24b912fb7e92172bf
  Stored in directory: /root/.cache/p

Для работы на TPU установим PyTorch-XLA

In [3]:
!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
!python pytorch-xla-env-setup.py --version 20200924  --apt-packages libomp5 libopenblas-dev

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  5116  100  5116    0     0  91357      0 --:--:-- --:--:-- --:--:-- 91357
Updating... This may take around 2 minutes.
Updating TPU runtime to pytorch-dev20200924 ...
     |████████████████████████████████| 61kB 3.0MB/s 
Uninstalling torch-1.6.0+cu101:
  Found existing installation: google-api-python-client 1.7.12
    Uninstalling google-api-python-client-1.7.12:
      Successfully uninstalled google-api-python-client-1.7.12
Done updating TPU runtime
  Successfully uninstalled torch-1.6.0+cu101
Uninstalling torchvision-0.7.0+cu101:
  Successfully uninstalled torchvision-0.7.0+cu101
Copying gs://tpu-pytorch/wheels/torch-nightly+20200924-cp36-cp36m-linux_x86_64.whl...
|
Operation completed over 1 objects/113.8 MiB.                                    
Copying gs://tpu-pytorch/wheels/torch_xla-nightly+20200924-cp36-cp36m-linux_x86

Импортируем необходимые библиотеки

In [4]:
import os
os.environ['XLA_USE_BF16'] = "1"
os.environ['XLA_TENSOR_ALLOCATOR_MAXSIZE'] = '100000000'

import gc
gc.enable()

import numpy as np
import pandas as pd

import transformers
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader #, Dataset
from torch.utils.data.distributed import DistributedSampler

import torch_xla
import torch_xla.core.xla_model as xm
import torch_xla.debug.metrics as met
import torch_xla.distributed.parallel_loader as pl
import torch_xla.distributed.xla_multiprocessing as xmp
import torch_xla.utils.serialization as xser
import torch_xla.version as xv

import warnings
warnings.filterwarnings("ignore")

Зафиксируем `seed` для воспроизводимости результата

In [5]:
import random

def my_seed(seed=42):
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)

my_seed()

## Model

Загружаем предобученную модель `roberta-large` и токенизатор

In [7]:
model = AutoModelForSequenceClassification.from_pretrained("roberta-large")
tokenizer = AutoTokenizer.from_pretrained("roberta-large")

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.weight', 'classif

Загрузим сохраненные веса модели (например, после 9-ой эпохи)

In [8]:
model.load_state_dict(torch.load('/content/drive/My Drive/v25/RoBERTa_9.pt'))

<All keys matched successfully>

In [9]:
WRAPPED_MODEL = xmp.MpModelWrapper(model)

## Data preprocessing

### Train & Val

Считываем данные для обучения

In [10]:
train_df = pd.read_csv('train_data.csv')

Колонки:
*  **Id** - уникальный идентификатор объявления
*  **Название** - название вакансии
*  **Место** - место работы (город, страна, штат)
*  **Отдел** - название отдела в компании, куда есть вакансия
*  **Зарплата** - предлагаемая зарплата на вакансию
*  **Описание компании** - описание компании, которая разместила вакансию. Составлено самой компанией
*  **Описание вакансии** - развернутое описание вакансии. Составлено компанией, которая разместила вакансию.
*  **Требования** - описание обязанностей работника
*  **Соцпакет** - все benefits, предоставляемые компанией работнику
*  **Дистанционно** - бинарная переменная: 1, если работа удаленная
*  **Вопросы** - бинарная переменная: 1, если в первичной анкете при приеме на работу присутствуют вопросы
*  **Тип занятости** - полный/неполный рабочий день, стажировка и т.д.
*  **Опыт** - опыт, которым должен обладать потенциальный кандидат
*  **Образование** - образование, которым должен обладать потенциальный кандидат
*  **Индустрия** - сфера деятельности компании. Ритейл/медицина и т.д.
*  **Позиция** - позиция, на которую открыта вакансия
*  **Фейк** - (только в тренировочном датасете) целевая переменная: 0, если объявление не фейк, 1, если фейк

In [11]:
train_df.head()

,Id,Название,Место,Отдел,Зарплата,Описание компании,Описание вакансии,Требования,Соцпакет,Дистанционно,Вопросы,Тип занятости,Опыт,Образование,Индустрия,Позиция,Фейк
0,10497,"Reporter, Agri Investor","US, ,",KHR,NaN,PEI Media (#URL_4cffe430b8da4c0d7e6074d11e638d...,Seeking savvy business reporter Are you a news...,The successful candidate should have a bachelo...,PEI Media (#URL_4cffe430b8da4c0d7e6074d11e638d...,0,0,Full-time,Associate,Bachelor's Degree,Publishing,Writing/Editing,0
1,8219,Software Engineer,"US, VA, Herndon",Software Engineering,NaN,Roka Security is a boutique security firm that...,Roka Security is looking for Software Engineer...,NaN,NaN,0,0,Full-time,Not Applicable,Unspecified,Computer Software,Engineering,0
2,5342,Structural Engineer,"NL, ZH, Rotterdam",onshore,NaN,NaN,Job tasks:Analyse project specifications on st...,Job requirements:HTS/TU Civil Engineering or M...,NaN,0,1,NaN,NaN,NaN,NaN,NaN,0
3,1916,Senior Publishing Specialist,"US, ,",NaN,NaN,NaN,The Senior Publishing Specialist will be part ...,Minimum of 3 – 4 years experience in Regulator...,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0
4,1811,Sales Representative,"CA, BC, Vancouver",Sales & Marketing,NaN,NaN,BCCC Enterprises provides management consultin...,Key Responsibilities:Contribute to the financi...,How to ApplyThis is an exciting opportunity. ...,0,1,NaN,NaN,NaN,NaN,NaN,0


Видим, что данные содержат достаточно много пустых значений

In [12]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10728 entries, 0 to 10727
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Id                 10728 non-null  int64 
 1   Название           10728 non-null  object
 2   Место              10504 non-null  object
 3   Отдел              3758 non-null   object
 4   Зарплата           1730 non-null   object
 5   Описание компании  8682 non-null   object
 6   Описание вакансии  10728 non-null  object
 7   Требования         9097 non-null   object
 8   Соцпакет           6397 non-null   object
 9   Дистанционно       10728 non-null  int64 
 10  Вопросы            10728 non-null  int64 
 11  Тип занятости      8586 non-null   object
 12  Опыт               6478 non-null   object
 13  Образование        5836 non-null   object
 14  Индустрия          7775 non-null   object
 15  Позиция            6853 non-null   object
 16  Фейк               10728 non-null  int64

Данные сильно несбалансированы: среди объявлений фейковых очень мало (примерно 5%)

In [13]:
train_df['Фейк'].value_counts()

0    10205
1      523
Name: Фейк, dtype: int64

Выделяются две колонки **Дистанционно** и **Вопросы**, в которых данные числовые. Представим их в строковом виде, заменив на подходящие по смыслу фразы, чтобы можно было подавать их на вход нашей модели наравне с остальными колонками.

In [14]:
train_df['Дистанционно'] = train_df['Дистанционно'].map({1:'Telecommuting positions', 0:'Office positions'})

In [15]:
train_df['Дистанционно'].value_counts()

Office positions           10274
Telecommuting positions      454
Name: Дистанционно, dtype: int64

In [16]:
train_df['Вопросы'] = train_df['Вопросы'].map({1:'Screening questions are present', 0:'There are no screening questions'})

In [17]:
train_df['Вопросы'].value_counts()

There are no screening questions    5424
Screening questions are present     5304
Name: Вопросы, dtype: int64

Выделим колонки, которые хотим подать на вход нашей модели

In [18]:
columns_to_text = ['Название', 
                   'Тип занятости',
                   'Дистанционно',
                   'Вопросы', 
                   'Описание компании', 
                   'Описание вакансии', 
                   'Требования']

Также соберем все колонки, кроме целевой переменной

In [19]:
old_columns = train_df.columns

In [20]:
old_columns = old_columns.drop('Фейк')

In [21]:
old_columns

Index(['Id', 'Название', 'Место', 'Отдел', 'Зарплата', 'Описание компании',
       'Описание вакансии', 'Требования', 'Соцпакет', 'Дистанционно',
       'Вопросы', 'Тип занятости', 'Опыт', 'Образование', 'Индустрия',
       'Позиция'],
      dtype='object')

Все пропуски в данных заменим на пустую строку

In [22]:
train_df.fillna('', inplace=True)

In [23]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10728 entries, 0 to 10727
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Id                 10728 non-null  int64 
 1   Название           10728 non-null  object
 2   Место              10728 non-null  object
 3   Отдел              10728 non-null  object
 4   Зарплата           10728 non-null  object
 5   Описание компании  10728 non-null  object
 6   Описание вакансии  10728 non-null  object
 7   Требования         10728 non-null  object
 8   Соцпакет           10728 non-null  object
 9   Дистанционно       10728 non-null  object
 10  Вопросы            10728 non-null  object
 11  Тип занятости      10728 non-null  object
 12  Опыт               10728 non-null  object
 13  Образование        10728 non-null  object
 14  Индустрия          10728 non-null  object
 15  Позиция            10728 non-null  object
 16  Фейк               10728 non-null  int64

Теперь объединим значения из выбранных колонок **columns_to_text** в одну строку и запишем ее в новую колонку **text_data**

In [24]:
train_df['text_data'] = train_df[columns_to_text].agg(' '.join, axis=1)

In [25]:
from pprint import pprint

Вот что у нас получилось. Не очень красиво, но "чистка" текста качество классификации только ухудшила (по крайней мере, у меня)

In [26]:
for i in range(10):
    pprint(train_df['text_data'][i])
    print()

('Reporter, Agri Investor Full-time Office positions There are no screening '
 'questions PEI Media\xa0'
 '(#URL_4cffe430b8da4c0d7e6074d11e638d28a1e165c8782b891e111fc17606c3d961#) '
 'provides a range of highly respected publications and market leading events '
 'to its financial sector audience. The\xa0leading information provider in '
 'private capital markets, PEI Media also has a strong international presence, '
 'with offices in London, New York and Hong Kong. Seeking savvy business '
 'reporter Are you a news-hungry journalist? Have a passion for finance, a '
 'talent for breaking news, networking and developing sophisticated sources?If '
 'so, there’s an exciting full-time reporter position available with PEI '
 'Media, the leading global B2B financial publishing and information group '
 'focused exclusively on alternative asset classes.Based in Manhattan, this '
 'opening is a rare opportunity for an ambitious individual seeking to advance '
 'their career in one of the most dy

Теперь можем разделить данные, выделив для валидации 1024 примера

In [27]:
from sklearn.model_selection import train_test_split

In [28]:
train_df, val_df = train_test_split(train_df, test_size=1024, random_state=42)

In [29]:
val_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1024 entries, 592 to 6311
Data columns (total 18 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Id                 1024 non-null   int64 
 1   Название           1024 non-null   object
 2   Место              1024 non-null   object
 3   Отдел              1024 non-null   object
 4   Зарплата           1024 non-null   object
 5   Описание компании  1024 non-null   object
 6   Описание вакансии  1024 non-null   object
 7   Требования         1024 non-null   object
 8   Соцпакет           1024 non-null   object
 9   Дистанционно       1024 non-null   object
 10  Вопросы            1024 non-null   object
 11  Тип занятости      1024 non-null   object
 12  Опыт               1024 non-null   object
 13  Образование        1024 non-null   object
 14  Индустрия          1024 non-null   object
 15  Позиция            1024 non-null   object
 16  Фейк               1024 non-null   int64

Удалим все старые колонки и наши валидационные данные готовы!

In [30]:
val_df.drop(columns=old_columns, inplace=True)

In [31]:
val_df

,Фейк,text_data
592,0,Technical Site Representative Full-time Offic...
608,0,Communications / Media Relations Full-time Of...
424,0,Customer Support Associate Office positions S...
10133,0,Property Manager (LCAM) Full-time Office posit...
5635,0,Talent Acquisition Specialist USA Full-time Of...
...,...,...
5269,0,Marketing Manager Full-time Office positions T...
7877,0,Healthcare Assistant Office positions There a...
4076,0,Clinical Psychotherapist Full-time Office posi...
106,0,Accountant / Office Manager Full-time Office p...


### Test

Повторяем те же действия с тестовым датасетом.

Прочитаем данные

In [39]:
test_df = pd.read_csv('test_data.csv')

In [40]:
test_df.head()

,Id,Название,Место,Отдел,Зарплата,Описание компании,Описание вакансии,Требования,Соцпакет,Дистанционно,Вопросы,Тип занятости,Опыт,Образование,Индустрия,Позиция
0,10728,Internet Marketing Specialist,"IE, L, Dublin",Marketing,NaN,NaN,This role represents an important position wit..., 3 to 10+ years experience in a similar inter...,RemunerationA highly attractive salary will be...,0,0,NaN,NaN,NaN,NaN,NaN
1,10729,Business Development,"DE, , Berlin",NaN,NaN,We are on a mission: Making cities smarter.Cit...,You have an opinion about how we can make our ...,NaN,NaN,0,1,NaN,NaN,NaN,NaN,NaN
2,10730,Sonographer - Full and Part Time Opportunities,"GB, ,",Ultrasound,NaN,Diagnostic Health deploy and manage high quali...,We are a leading local ultrasound company spec...,Diploma of College of Radiographers or BSc (Ho...,Competative SalaryHours to suit,0,1,Part-time,NaN,Associate Degree,Medical Practice,Health Care Provider
3,10731,Care Support Worker,"GB, EDH, Edinburgh",NaN,NaN,Social Care Alba is the highest graded Care at...,Social Care Alba is the highest quality Care a...,Key Accountabilities would be:To provide indiv...,This is your chance to become part of our cont...,0,1,NaN,Not Applicable,NaN,Hospital & Health Care,NaN
4,10732,Outside Sales Professional-Springfield,"US, IL, Springfield",NaN,NaN,"ABC Supply Co., Inc. is the nation’s largest w...","As an Outside Sales Representative, you must h...",Track Record of Sales Success – B2B or B2CNo m...,"As an Outside Sales Representative, you will r...",0,0,NaN,NaN,NaN,NaN,NaN


In [41]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7152 entries, 0 to 7151
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Id                 7152 non-null   int64 
 1   Название           7152 non-null   object
 2   Место              7030 non-null   object
 3   Отдел              2575 non-null   object
 4   Зарплата           1138 non-null   object
 5   Описание компании  5890 non-null   object
 6   Описание вакансии  7151 non-null   object
 7   Требования         6088 non-null   object
 8   Соцпакет           4273 non-null   object
 9   Дистанционно       7152 non-null   int64 
 10  Вопросы            7152 non-null   int64 
 11  Тип занятости      5823 non-null   object
 12  Опыт               4352 non-null   object
 13  Образование        3939 non-null   object
 14  Индустрия          5202 non-null   object
 15  Позиция            4572 non-null   object
dtypes: int64(3), object(13)
memory usage: 894.

Преобразуем значения в колонках **Дистанционно** и **Вопросы**

In [42]:
test_df['Дистанционно'] = test_df['Дистанционно'].map({1:'Telecommuting positions', 0:'Office positions'})

In [43]:
test_df['Вопросы'] = test_df['Вопросы'].map({1:'Screening questions are present', 0:'There are no screening questions'})

Заменим пропуски

In [44]:
test_df.fillna('', inplace=True)

Добавим новую колонку с объединенным текстом из выбранных колонок

In [45]:
test_df['text_data'] = test_df[columns_to_text].agg(' '.join, axis=1)

Удалим все старые колонки

In [46]:
test_df.drop(columns=old_columns, inplace=True)

In [47]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7152 entries, 0 to 7151
Data columns (total 1 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   text_data  7152 non-null   object
dtypes: object(1)
memory usage: 56.0+ KB


## Datasets

С помощью токенизатора преобразуем наши тексты в вектора `input_ids` одной длины (512) и переведем их в необходимый для нашей модели формат тензоров `torch`

In [48]:
import nlp

In [49]:
MAX_LEN = 512 

def convert_to_features(batch):

    encodings = tokenizer.batch_encode_plus(batch['text_data'], 
                                            add_special_tokens=True,
                                            pad_to_max_length=True,
                                            truncation=True,
                                            max_length=MAX_LEN)
    return encodings

valid_dataset = nlp.Dataset.from_pandas(val_df)
valid_dataset = valid_dataset.map(convert_to_features, batched=True)  
valid_dataset.set_format("torch", columns=['input_ids', 'attention_mask', 'Фейк'])

test_dataset = nlp.Dataset.from_pandas(test_df) 
test_dataset = test_dataset.map(convert_to_features, batched=True)  
test_dataset.set_format("torch", columns=['input_ids', 'attention_mask'])

## Evaluate & Submit

### Evaluation

Опишем функцию для валидации

In [ ]:
from sklearn.metrics import f1_score

In [53]:
from sklearn.metrics import confusion_matrix

In [54]:
def eval_loop_fn(test_data_loader, model, device):

    model.eval()

    with torch.no_grad():

        test_preds = []
        targets = []
        val_loss = 0

        for i, data in enumerate(test_data_loader):

            input_ids = data["input_ids"].to(device, dtype=torch.long)
            attention_mask = data["attention_mask"].to(device, dtype=torch.long)
            labels = data["Фейк"].to(device, dtype=torch.float)

            outputs = model(
                input_ids = input_ids,
                attention_mask = attention_mask,
                labels = labels
            )

            val_loss += outputs[0].item()
            test_preds.extend(torch.argmax(outputs[1], -1).cpu().detach().tolist())
            targets.extend(data["Фейк"].detach().tolist())  
        
    xm.master_print(f'Val loss: {val_loss / len(test_data_loader)}')
    xm.master_print(f'F1 score: {f1_score(targets, test_preds)}')
    xm.master_print('Confusion matrix:')
    xm.master_print(f'{confusion_matrix(targets, test_preds)}')

    del outputs
    del targets
    del val_loss
    del test_preds
    del input_ids
    del attention_mask
    del labels
    gc.collect()

### Test

Опишем функцию, которая будет получать предсказания для тестового датасета и записывать их в колонку `Фейк` файла `sample_submission.csv` под именем `submission_epoch.csv`

In [55]:
sample_submission = pd.read_csv('sample_submission.csv')

In [60]:
def test_loop_fn(test_data_loader, model, device):

    model.eval()

    with torch.no_grad():

        test_preds = []

        for i, data in enumerate(test_data_loader):

            input_ids = data["input_ids"].to(device, dtype=torch.long)
            attention_mask = data["attention_mask"].to(device, dtype=torch.long)

            outputs = model(
                input_ids = input_ids,
                attention_mask = attention_mask,
            )[0]
            
            test_preds.extend(torch.argmax(outputs, -1).cpu().detach().tolist())  

    sample_submission['Фейк'] = test_preds
    sample_submission.to_csv(f'submission.csv', index=False)

    del outputs
    del test_preds
    del input_ids
    del attention_mask
    gc.collect()

### Run

Укажем параметры обучения

In [57]:
my_seed(42)

TRAIN_BATCH_SIZE = 4 * xm.xrt_world_size()
VALID_BATCH_SIZE = 8 
TEST_BATCH_SIZE = 8
EPOCHS = 10
LR = 5e-5 * xm.xrt_world_size() 

Определим загрузчики данных для каждого датасета и опишем основной цикл обучения. После каждой эпохи будем оценивать качество на валидационном датасете и делать предсказания для тестового.

In [61]:
def _run():
    
    # Valid    
    valid_data_loader = DataLoader(
        valid_dataset,
        batch_size=VALID_BATCH_SIZE,
        drop_last=False,
        num_workers=0,
        shuffle=False,
    )
    xm.master_print('Valid Loader Created.')

    # Test
    test_data_loader = DataLoader(
        test_dataset, 
        batch_size=TEST_BATCH_SIZE,
        drop_last=False,
        num_workers=0,
        shuffle=False,
    )
    xm.master_print('Test Loader Created.')

    device = xm.xla_device()
    model = WRAPPED_MODEL.to(device)
    xm.master_print('Done Model Loading.')
            
    para_loader = pl.ParallelLoader(valid_data_loader, [device])
    xm.master_print('Parallel Loader Created. Validating ...')
    eval_loop_fn(para_loader.per_device_loader(device), 
                model,  
                device
                )
        
    para_loader = pl.ParallelLoader(test_data_loader, [device])
    xm.master_print('Parallel Loader Created. Testing ...')
    test_loop_fn(para_loader.per_device_loader(device), 
                model,  
                device, 
                )
    xm.master_print('Predicted.')

Запускаем обучение

In [62]:
def _mp_fn(rank, flags):
    _run()

FLAGS={}
xmp.spawn(_mp_fn, args=(FLAGS,), nprocs=8, start_method='fork')

Valid Loader Created.
Test Loader Created.
Done Model Loading.
Parallel Loader Created. Validating ...
Val loss: 0.051411598920822144
F1 score: 0.8928571428571428
Confusion matrix:
[[962   5]
 [  7  50]]
Parallel Loader Created. Testing ...
Predicted.
